# 事前準備
## ライブラリのインポート

In [10]:
from gurobipy import Model, quicksum, GRB
import pandas as pd
import pickle

## 事前に用意されたデータの読み込み
読み込んでいる`logi_data.dump`ファイルは、同ディレクトリ内にある`make_logi_data.ipynb`ファイルで作成しています。

In [11]:
with open('logi_data.dump', 'rb') as f:
    Cust = pickle.load(f)                #顧客の集合
    DC = pickle.load(f)                   #流通センターの集合
    Route = pickle.load(f)              #配送ルートの集合
    CT = pickle.load(f)                   #センタータイプの集合
    transport = pickle.load(f)       #輸送費
    delivery = pickle.load(f)          #配送費
    dc_run = pickle.load(f)           #流通センター維持費
    dc_stock = pickle.load(f)       #流通センター在庫費
    dc_ub = pickle.load(f)            #流通センター保管容量
    dc_new = pickle.load(f)         #流通センター新規契約費
    dc_cancel = pickle.load(f)     #流通センター解約費
    dc_disposal = pickle.load(f)  #流通センター在庫廃棄費
    cust_stock = pickle.load(f)   #販社在庫費
    cust_out = pickle.load(f)       #販社品切れ費

## 計画期間の決定

In [12]:
T = 8                                             #モデルの計画期の決定
Period = list(range(1, T+1))      #計画期間のリスト

## demand.csvファイルを読み込み

In [13]:
df = pd.read_csv('demand.csv', header=None)
demand = {(j,t):df[t-1][j-1] for j in Cust for t in Period}

# モデルの構築
## モデルの作成

In [14]:
model = Model()

## 変数の追加

In [15]:
X, x, y, sD, sC, z, w, o, d = {}, {}, {}, {}, {}, {}, {}, {}, {}

#期tにおける流通センターiへの輸送量X[i,t]を追加
for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="I", name=f'X[{i},{t}]')

#期tにおける流通センターiから販社jへの配送量x[i,j,t]を追加
for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="I", name=f'x[{i},{j},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを運用するかどうかの0-1変数y[i,ct,t]を追加
for i in DC:
    for ct in CT:
        y[i,ct,0] = model.addVar(vtype="B", ub=0, name=f'y[{i},{ct},{0}]')
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

#期tにおける流通センターiの在庫量sD[i,t]を追加
for i in DC:
    sD[i,0] = model.addVar(vtype="I", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="I", name=f'sD[{i},{t}]')
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

#期tにおける販社jの在庫量sC[j,t]を追加
for j in Cust:
    sC[j,0] = model.addVar(vtype="I", ub=0, name=f'sC[{j},{t}]')
    for t in Period:
        sC[j,t] = model.addVar(vtype="I", name=f'sC[{j},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを新しく建てるかどうかのバイナリ変数z[i,ct,t]を追加
for i in DC:
    for ct in CT:
        for t in Period:
            z[i,ct,t] = model.addVar(vtype="B", name=f'z[{i},{ct},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを解約するかどうかのバイナリ変数w[i,ct,t]を追加
for i in DC:
    for ct in CT:
        for t in Period:
            w[i,ct,t] = model.addVar(vtype="B", name=f'w[{i},{ct},{t}]')

#期tにおける販社jの品切れ量o[j,t]を追加
for j in Cust:
    for t in Period:
        o[j,t] = model.addVar(vtype="I", name=f'o[{j},{t}]')

#期tにおける流通センターiの在庫量廃棄量d[i,t]を追加
for i in DC:
    for t in Period:
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

        
model.update()

## 制約の追加

In [18]:
Cust_Demand_Cons, DC_Flow_Cons, DC_Running_Cons, CT_Only_Cons, DC_UB_Cons, DC_Connect_Cons = {}, {}, {}, {}, {}, {}

#期tにおける販社jの需要を満たすための需要制約
for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC) + sC[j,t-1]
            ==
            demand[j,t] + sC[j,t] - o[j,t]
        )

#期tにおける、流通センターiのフロー整合条件
for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t] + d[i,t]
        )

#流通センターに関する0-1変数y[i,ct,t]を一意に定めるための強化制約
for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

#期tにおいて地点iで選択できる流通センターのタイプは１つまで
for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

#期tにおける、流通センターi、センタータイプctの容量上限制約
for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )

#新規契約と解約をしたかどうかを一意に定める制約
for i in DC:
    for ct in CT:
        for t in Period:
            DC_Connect_Cons[i,ct,t] = model.addConstr(
                y[i,ct,t] - y[i,ct,t-1]
                ==
                z[i,ct,t] - w[i,ct,t]
            )
    
model.update()

## 目的関数の設定

In [19]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +                               #輸送費
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +                      #配送費
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +     #流通センター維持費
    quicksum(dc_stock * sD[i,t] for i in DC for t in Period) +                                #流通センター在庫費
    quicksum(cust_stock * sC[j,t] for j in Cust for t in Period) +                          #販社在庫費
    quicksum(dc_new * z[i,ct,t] for i in DC for ct in CT for t in Period) +             #流通センター新規契約費 
    quicksum(dc_cancel * w[i,ct,t] for i in DC for ct in CT for t in Period) +        #流通センター解約費
    quicksum(cust_out * o[j,t] for j in Cust for t in Period) +                                #販社品切れ費
    quicksum(dc_disposal * d[i,t] for i in DC for t in Period)                                  #流通センター廃棄費
    ,GRB.MINIMIZE
)

model.update()

## 求解

In [20]:
model.optimize()

Optimize a model with 1760 rows, 2893 columns and 11176 nonzeros
Variable types: 0 continuous, 2893 integer (1375 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 354200.00000
Presolve removed 132 rows and 308 columns
Presolve time: 0.02s
Presolved: 1628 rows, 2585 columns, 10483 nonzeros
Variable types: 0 continuous, 2585 integer (1210 binary)

Root relaxation: objective 8.113753e+04, 1223 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 81137.5261    0  145 354200.000 81137.5261  77.1%     -    0s
H    0     0                    271980.40000 81137.5261  70.2%     -    0s
H    0     0                    214480.80000 81137.5261  62.2%     -    0s
H    0     0                    176880.40000 8113

# 最適解の可視化

In [21]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[7,1] 80.0
X[7,2] 80.0
X[7,3] 80.0
X[7,4] 72.0
X[7,5] 78.0
X[7,6] 71.0
X[7,7] 80.0
X[7,8] 80.0
X[9,5] 59.0
X[9,6] 80.0
X[9,7] 80.0
X[9,8] 80.0
X[10,1] 80.0
X[10,2] 80.0
X[10,3] 80.0
X[10,4] 80.0
X[10,5] 80.0
X[10,6] 80.0
X[10,7] 80.0
X[10,8] 80.0
X[11,6] 57.0
X[11,7] 68.0
X[11,8] 80.0
x[11,11,6] 32.0
x[11,11,7] 40.0
x[11,11,8] 41.0
x[11,5,6] 25.0
x[11,5,7] 14.0
x[11,5,8] 19.0
x[7,6,1] 20.0
x[7,6,2] 18.0
x[7,6,3] 15.0
x[7,6,4] 10.0
x[7,6,5] 22.0
x[7,6,6] 33.0
x[7,6,7] 36.0
x[7,6,8] 30.0
x[9,9,5] 30.0
x[9,9,6] 47.0
x[9,9,7] 39.0
x[9,9,8] 34.0
x[7,8,1] 16.0
x[7,8,2] 18.0
x[7,8,3] 25.0
x[7,8,4] 10.0
x[7,8,6] 2.0
x[10,3,1] 12.0
x[10,3,2] 15.0
x[10,3,3] 7.0
x[10,3,4] 11.0
x[10,3,5] 6.0
x[10,3,6] 12.0
x[10,3,7] 18.0
x[10,3,8] 23.0
x[7,2,1] 12.0
x[7,2,2] 9.0
x[7,2,3] 6.0
x[7,2,4] 14.0
x[7,2,5] 15.0
x[10,9,1] 8.0
x[10,9,2] 16.0
x[10,9,3] 14.0
x[11,4,7] 4.0
x[11,4,8] 5.0
x[10,4,1] 8.0
x[10,4,2] 9.0
x[10,4,3] 7.0
x[10,4,4] 9.0
x[10,4,5] 8.0
x[10,4,6] 14.0
x[10,4,7] 10.0
x[7,1,1] 5.0
x[10,10,1] 2

In [37]:
TermObj = {t:   sum(transport[i] * X[i,t] for i in DC) +
                           sum(delivery[i,j] * x[i,j,t] for i,j in Route) +                      
                           quisum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT) +
                           quicksum(dc_stock * sD[i,t] for i in DC) +                                
                           quicksum(cust_stock * sC[j,t] for j in Cust) +  
                           quicksum(dc_new * z[i,ct,t] for i in DC for ct in CT) +          
                           quicksum(dc_cancel * w[i,ct,t] for i in DC for ct in CT) +
                           quicksum(cust_out * o[j,t] for j in Cust) +                             
                           quicksum(dc_disposal * d[i,t] for i in DC) for t in Period}

In [36]:
print('-*-*-*-*-*-*- RESULT -*-*-*-*-*-*-')
print(f'Total Cost: {model.ObjVal}')

for t in Period:
        print(f'Term {t}')
        print(f'Obj  {t}: {TermObj[t]}')
        print(' Center [place: type]')
        print('  New   : {0}'.format({int(float(i)):int(float(k)) for i in I for k in K if z[i,k,t].X >= 1}))
        print('  Use   : {0}'.format({int(float(i)):int(float(k)) for i in I for k in K if y[i,k,t].X >= 1}))
        print('  Cancel: {0}'.format({int(float(i)):int(float(k)) for i in I for k in K if w[i,k,t].X >= 1}))
        print(' Transport')
        for i in I:
            if X[i,t].X > 1e-5:
                print('  to Center {0:>2d}: {1:>5s}'.format(int(float(i)),str(X[i,t].X)))
        for j in J:
            for i in I:
                if x[i,j,t].X > 1e-5:
                    print('  to Distributor {0:>2d} from Center {1:>2d}: {2:>5s}'.format(int(float(j)),int(float(i)),str(x[i,j,t].X)))
        print(' Inventories')
        for i in I:
            cStock = False
            if sC[i,t].X > 1e-05:
                cStock = True
                print('  Center {0:>2d}: {1:>5s}'.format(int(float(i)), str(sC[i,t].X)))
        if not cStock:
            print('  Center     : None')
        for j in J:
            dStock = False
            if sD[j,t].X > 1e-05:
                dStock = True
                print('  Distributor {0:>2d}: {1:>5s}'.format(int(float(j)), str(sD[j,t].X)))
        if not dStock:
            print('  Distributor: None')
        print(' Lost Sales')
        for j in J:
            lSales = False
            if xi[j,t].X > 1e-05:
                lSales = True
                print('  Distributor {0:>2d}: {1:>5s}'.format(int(float(j)), str(xi[j,t].X)))
        if not lSales:
            print('  None')
        print(' Disposal')
        for i in I:
            disposal = False
            if theta[i,t].X > 1e-05:
                disposal = True
                print('  Center {0:>2d}: {1:>5s}'.format(int(float(i)), str(theta[i,t].X)))
        if not disposal:
            print('  None')

-*-*-*-*-*-*- RESULT -*-*-*-*-*-*-
Total Cost: 93384.8
Term 1
Obj  1: <gurobi.LinExpr: 15.2 X[1,1] + 14.8 X[2,1] + 15.8 X[3,1] + 12.6 X[4,1] + 13.4 X[5,1] + 15.8 X[6,1] + 16.2 X[7,1] + 16.4 X[8,1] + 16.4 X[9,1] + 14.6 X[10,1] + 15.4 X[11,1] + 22.0 x[7,3,1] + 32.0 x[6,9,1] + 0.0 x[11,11,1] + 18.0 x[1,6,1] + 22.0 x[3,7,1] + 22.0 x[2,5,1] + 18.0 x[1,11,1] + 34.0 x[8,5,1] + 34.0 x[5,8,1] + 22.0 x[10,8,1] + 16.0 x[6,7,1] + 0.0 x[5,5,1] + 12.0 x[11,5,1] + 28.0 x[10,7,1] + 16.0 x[7,6,1] + 32.0 x[6,10,1] + 0.0 x[1,1,1] + 10.0 x[4,10,1] + 8.0 x[3,2,1] + 20.0 x[2,6,1] + 12.0 x[8,2,1] + 12.0 x[5,11,1] + 16.0 x[4,5,1] + 12.0 x[9,3,1] + 30.0 x[7,5,1] + 16.0 x[3,1,1] + 30.0 x[2,11,1] + 0.0 x[9,9,1] + 12.0 x[7,8,1] + 28.0 x[3,11,1] + 12.0 x[2,1,1] + 8.0 x[8,9,1] + 24.0 x[9,4,1] + 8.0 x[5,1,1] + 6.0 x[10,3,1] + 14.0 x[7,2,1] + 26.0 x[11,10,1] + 8.0 x[1,5,1] + 28.0 x[3,6,1] + 0.0 x[2,2,1] + 18.0 x[1,10,1] + 26.0 x[8,6,1] + 12.0 x[4,1,1] + 16.0 x[10,9,1] + 20.0 x[9,7,1] + 30.0 x[6,4,1] + 16.0 x[5,4,1] +

NameError: name 'I' is not defined

In [34]:
TermObj[1]

<gurobi.LinExpr: 15.2 X[1,1] + 14.8 X[2,1] + 15.8 X[3,1] + 12.6 X[4,1] + 13.4 X[5,1] + 15.8 X[6,1] + 16.2 X[7,1] + 16.4 X[8,1] + 16.4 X[9,1] + 14.6 X[10,1] + 15.4 X[11,1] + 22.0 x[7,3,1] + 32.0 x[6,9,1] + 0.0 x[11,11,1] + 18.0 x[1,6,1] + 22.0 x[3,7,1] + 22.0 x[2,5,1] + 18.0 x[1,11,1] + 34.0 x[8,5,1] + 34.0 x[5,8,1] + 22.0 x[10,8,1] + 16.0 x[6,7,1] + 0.0 x[5,5,1] + 12.0 x[11,5,1] + 28.0 x[10,7,1] + 16.0 x[7,6,1] + 32.0 x[6,10,1] + 0.0 x[1,1,1] + 10.0 x[4,10,1] + 8.0 x[3,2,1] + 20.0 x[2,6,1] + 12.0 x[8,2,1] + 12.0 x[5,11,1] + 16.0 x[4,5,1] + 12.0 x[9,3,1] + 30.0 x[7,5,1] + 16.0 x[3,1,1] + 30.0 x[2,11,1] + 0.0 x[9,9,1] + 12.0 x[7,8,1] + 28.0 x[3,11,1] + 12.0 x[2,1,1] + 8.0 x[8,9,1] + 24.0 x[9,4,1] + 8.0 x[5,1,1] + 6.0 x[10,3,1] + 14.0 x[7,2,1] + 26.0 x[11,10,1] + 8.0 x[1,5,1] + 28.0 x[3,6,1] + 0.0 x[2,2,1] + 18.0 x[1,10,1] + 26.0 x[8,6,1] + 12.0 x[4,1,1] + 16.0 x[10,9,1] + 20.0 x[9,7,1] + 30.0 x[6,4,1] + 16.0 x[5,4,1] + 16.0 x[11,4,1] + 10.0 x[10,4,1] + 22.0 x[7,1,1] + 34.0 x[6,11,1] + 40